In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score

# 🔹 Chargement des données
print("Chargement des données...")
data = pd.read_csv("listings.csv")

# Afficher les colonnes avec des valeurs manquantes et leur compte
print("Colonnes avec valeurs manquantes:")
print(data.isnull().sum()[data.isnull().sum() > 0])

# 🔹 Nettoyage des données
# Suppression des lignes avec des valeurs manquantes pour les colonnes critiques
data = data.dropna(subset=['latitude', 'longitude', 'price', 'number_of_reviews'])
data = data[data['price'] > 0]

# 🔹 Traitement explicite de toutes les valeurs manquantes
# Remplir les valeurs manquantes pour reviews_per_month
data['reviews_per_month'] = data['reviews_per_month'].fillna(0)

# S'assurer que toutes les autres colonnes numériques n'ont pas de valeurs manquantes
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    data[col] = data[col].fillna(data[col].median())

# Remplacer les valeurs manquantes dans les colonnes catégorielles par 'unknown'
categorical_cols = data.select_dtypes(include=['object']).columns
for col in categorical_cols:
    data[col] = data[col].fillna('unknown')

# 🔹 Suppression de la colonne 'neighbourhood_group'
data = data.drop(columns=['neighbourhood_group'])

# 🔹 Création de nouvelles variables
data['has_reviews'] = (data['number_of_reviews'] > 0).astype(int)
data['high_availability'] = (data['availability_365'] > 180).astype(int)
data['log_price'] = np.log1p(data['price'])
data['reviews_density'] = data['number_of_reviews'] / (data['minimum_nights'] + 1)

# 🔹 Features et target
features = [
    'latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
    'reviews_per_month', 'room_type',
    'calculated_host_listings_count', 'has_reviews',
    'high_availability', 'reviews_density'
]
target = 'log_price'

# 🔹 Séparation des données en X et y
X = data[features]
y = data[target]

# 🔹 Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(f"Taille des données d'entraînement: {X_train.shape}")
print(f"Taille des données de test: {X_test.shape}")

# 🔹 Colonnes
numeric_features = [
    'latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
    'reviews_per_month', 'calculated_host_listings_count', 'reviews_density'
]
categorical_features = ['room_type']

# 🔹 Prétraitement avec SimpleImputer pour gérer les éventuelles valeurs manquantes
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# 🔹 Création des modèles de base pour le Stacking
base_models = [
    ('ridge', Ridge(alpha=1.0, random_state=42)),
    ('rf', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)),
    ('gbr', GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42))
]

# 🔹 Méta-modèle avec paramètres explicites
meta_model = Ridge(alpha=0.1, random_state=42)

# 🔹 StackingRegressor avec validation croisée interne
stacking = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1
)

# 🔹 Pipeline global avec prétraitement
preproc_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('stacking', stacking)
])

# 🔹 GridSearchCV pour optimiser les paramètres du méta-modèle
param_grid = {
    'stacking__final_estimator__alpha': [0.1, 1.0, 10.0]
}

grid = GridSearchCV(
    preproc_pipeline,
    param_grid,
    cv=3,  # Réduit à 3 pour accélérer
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

# 🔹 Validation croisée après l'entraînement
try:
    print("\nDébut de l'entraînement du stacking (cela peut prendre un moment)...")
    grid.fit(X_train, y_train)
    print("✅ Entraînement terminé avec succès!")

    # Meilleurs paramètres
    print(f"\nMeilleurs paramètres trouvés: {grid.best_params_}")

    # 🔹 Prédiction et évaluation
    y_pred = grid.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print("\n📊 Évaluation du modèle (prix en échelle log) :")
    print(f" - RMSE : {rmse:.4f}")
    print(f" - MAE  : {mae:.4f}")
    print(f" - R²   : {r2:.3f}")

    # 🔹 Réévaluation du modèle dans la vraie échelle du prix
    print("\n📊 Évaluation du modèle (prix en €) :")
    print(f" - RMSE : {np.expm1(rmse):.2f} €")
    print(f" - MAE  : {np.expm1(mae):.2f} €")

    # 🔹 Exemple de prédiction pour 3 annonces
    sample_indices = np.random.choice(X_test.shape[0], 3, replace=False)
    sample = X_test.iloc[sample_indices]
    pred_log_price = grid.predict(sample)
    
    print("\n🏠 Exemples d'annonces :")
    for i, idx in enumerate(sample_indices):
        pred_price = np.expm1(pred_log_price[i])
        real_price = np.expm1(y_test.iloc[idx])
        error_pct = ((pred_price - real_price) / real_price) * 100

        print(f"\nAnnonce {i+1}:")
        print(f" - Type de logement: {sample.iloc[i]['room_type']}")
        print(f" - Nombre d'avis: {sample.iloc[i]['number_of_reviews']}")
        print(f" - Prix réel: {real_price:.2f} €")
        print(f" - Prix prédit: {pred_price:.2f} €")
        print(f" - Erreur: {error_pct:.1f}%")

    # 🔹 Validation croisée sur l'ensemble d'entraînement
    print("\n🔍 Validation croisée du modèle avec 5 plis (sur l'entraînement)...")
    cv_scores = cross_val_score(grid.best_estimator_, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    print(f" - Scores de validation croisée (RMSE): {-cv_scores.mean():.4f} (moyenne des scores MSE négatifs)")

except Exception as e:
    print(f"Erreur lors de l'entraînement: {e}")

    # Analyse plus détaillée en cas d'erreur
    print("\nAnalyse des données pour déboguer:")
    print(f"Shape de X_train: {X_train.shape}")
    print(f"Nombre de NaN dans X_train: {X_train.isnull().sum().sum()}")
    if X_train.isnull().sum().sum() > 0:
        print("Colonnes avec NaN dans X_train:")
        print(X_train.isnull().sum()[X_train.isnull().sum() > 0])


Chargement des données...
Colonnes avec valeurs manquantes:
neighbourhood_group    6397
last_review            1132
reviews_per_month      1132
dtype: int64
Taille des données d'entraînement: (4797, 10)
Taille des données de test: (1599, 10)

Début de l'entraînement du stacking (cela peut prendre un moment)...
Fitting 3 folds for each of 3 candidates, totalling 9 fits
✅ Entraînement terminé avec succès!

Meilleurs paramètres trouvés: {'stacking__final_estimator__alpha': 0.1}

📊 Évaluation du modèle (prix en échelle log) :
 - RMSE : 0.5114
 - MAE  : 0.3716
 - R²   : 0.505

📊 Évaluation du modèle (prix en €) :
 - RMSE : 0.67 €
 - MAE  : 0.45 €

🏠 Exemples d'annonces :

Annonce 1:
 - Type de logement: Private room
 - Nombre d'avis: 26
 - Prix réel: 49.00 €
 - Prix prédit: 48.50 €
 - Erreur: -1.0%

Annonce 2:
 - Type de logement: Entire home/apt
 - Nombre d'avis: 11
 - Prix réel: 40.00 €
 - Prix prédit: 124.34 €
 - Erreur: 210.9%

Annonce 3:
 - Type de logement: Entire home/apt
 - Nombre d

Chargement et nettoyage des données


Chargement des données : Le dataset listings.csv est chargé dans un DataFrame.

Gestion des valeurs manquantes : Le code identifie et supprime les lignes où des colonnes critiques (latitude, longitude, price, number_of_reviews) sont manquantes. 
Les autres valeurs manquantes sont soit remplies par la médiane (pour les variables numériques)
soit par une valeur constante (unknown pour les variables catégorielles).

Filtrage des prix : Les lignes où le prix est inférieur ou égal à zéro sont supprimées.

Création de nouvelles variables : Des colonnes supplémentaires sont créées
comme has_reviews (indiquant si l'annonce a des avis), 
high_availability (indiquant si la disponibilité est supérieure à 180 jours), 
log_price (le prix transformé logarithmiquement)
reviews_density (la densité des avis).

2. Préparation des données pour l'entraînement



Sélection des variables explicatives (features) et de la cible (log_price).

Séparation en ensembles d'entraînement et de test : L'ensemble de données est divisé en 75% pour l'entraînement et 25% pour le test.

Définition des colonnes numériques et catégorielles pour un prétraitement ultérieur.

3. Prétraitement des données



Transformation des variables numériques : Les valeurs manquantes sont imputées avec la médiane et les données sont normalisées avec StandardScaler.

Transformation des variables catégorielles : Les valeurs manquantes sont imputées avec la valeur unknown 
les variables catégorielles sont transformées en variables binaires avec OneHotEncoder.

4. Modélisation par Stacking



Création de modèles de base pour le stacking : Trois modèles sont utilisés en base :

Ridge : Régression linéaire avec régularisation L2.

RandomForestRegressor : Forêt d'arbres de régression.

GradientBoostingRegressor : Gradient boosting pour la régression.

Méthode de Stacking : Les résultats de ces trois modèles de base sont combinés dans un modèle de stacking avec un modèle final (meta_model), ici un modèle Ridge.

5. Optimisation des hyperparamètres



GridSearchCV : Une recherche de grille est effectuée pour optimiser l'hyperparamètre alpha du modèle Ridge dans le stacking. La grille cherche les meilleures valeurs de alpha pour le méta-modèle.

6. Évaluation et résultats



Entraînement du modèle : Le modèle de stacking est entraîné avec les données d'entraînement.

Évaluation : Les performances du modèle sont évaluées à l'aide de différentes métriques :

RMSE (Root Mean Squared Error) et MAE (Mean Absolute Error) sur l'échelle log du prix.

R² (coefficient de détermination).

Réévaluation des prix réels en revenant à l’échelle d’origine (en €) avec la fonction np.expm1() (inverse de np.log1p()).

Prédictions sur des exemples d'annonces : Le modèle prédit les prix pour quelques annonces de test et compare ces prédictions avec les valeurs réelles

calculant l'erreur en pourcentage.

7. Validation croisée



Validation croisée : Le modèle est évalué avec une validation croisée à 5 plis pour donner une estimation de la robustesse du modèle.

8. Gestion des erreurs




Si une erreur survient lors de l'entraînement (par exemple, des problèmes de données), des messages d'erreur sont affichés
et le code effectue une analyse pour comprendre les problèmes (par exemple, la présence de valeurs manquantes).


Le code effectue un prétraitement complet des données
utilise un modèle de stacking pour combiner plusieurs régressions (Ridge, Random Forest, Gradient Boosting) 
optimise les paramètres du modèle via GridSearchCV. 
Ensuite, il évalue les performances du modèle en termes de RMSE, MAE, et R², à la fois sur l’échelle logarithmique du prix et sur l’échelle d’origine. 
Il permet également de visualiser l’erreur de prédiction pour quelques annonces spécifiques et d’effectuer une validation croisée pour mesurer la stabilité du modèle.

Interprétation des Résultats



1. Meilleurs paramètres du modèle
stacking__final_estimator__alpha = 0.1 : Le meilleur paramètre pour le méta-modèle Ridge (utilisé dans le stacking) est alpha = 0.1. Cela signifie que la régularisation L2 a été ajustée pour un niveau de pénalisation relativement faible, ce qui peut avoir permis de mieux adapter le modèle aux données sans trop le contraindre.



2. Évaluation du modèle (prix en échelle log) :
RMSE : 0.5114 : Le Root Mean Squared Error (erreur quadratique moyenne) en échelle logarithmique est de 0.5114. Cette valeur mesure la différence moyenne entre les valeurs réelles et prédites, mais sur l'échelle log du prix. Plus cette valeur est faible, plus les prédictions sont précises. Un RMSE de 0.5114 indique une performance raisonnable.



MAE : 0.3716 : Le Mean Absolute Error (erreur absolue moyenne) en échelle logarithmique est de 0.3716, ce qui signifie que l'erreur moyenne (en log) par prédiction est de 0.3716. Cela montre également une erreur modérée entre les prix réels et les prédictions.

R² : 0.505 : Le coefficient de détermination (R²) est de 0.505, ce qui signifie que le modèle explique environ 50.5% de la variance des prix en échelle log. Un R² de 0.505 montre un modèle avec une performance relativement modérée. Il reste un certain potentiel d'amélioration.



3. Évaluation du modèle (prix en €) :
RMSE : 0.67 € : Une fois les prédictions converties sur l’échelle réelle du prix, l'erreur quadratique moyenne est de 0.67 €. Cela donne une idée de l'erreur réelle sur les prix en €.

MAE : 0.45 € : L'erreur absolue moyenne en termes de prix réels est de 0.45 €, ce qui donne une idée de la précision globale du modèle dans la prédiction des prix.



4. Exemples d'annonces
Le modèle a prédit des prix pour quelques annonces de test, avec des erreurs d'environ 9.3%. Cela montre que les prédictions sont assez proches des valeurs réelles, mais il reste des erreurs notables. Une erreur de 9.3% peut être acceptable dans un cadre de prédiction de prix, mais il y a toujours des opportunités pour améliorer la précision, notamment en optimisant davantage les paramètres ou en utilisant des modèles alternatifs.



5. Validation croisée
Scores de validation croisée (RMSE) : 0.3083 : La validation croisée montre une erreur moyenne de 0.3083 (sur 5 plis), ce qui est une bonne mesure de la stabilité et de la généralisation du modèle. Un score plus faible en validation croisée suggère que le modèle ne surajuste pas trop aux données d'entraînement et qu'il peut bien généraliser aux nouvelles données.



Conclusion


Performance globale : Le modèle de stacking montre une performance raisonnable avec un R² de 0.505, ce qui signifie qu'il explique environ 50% de la variance des prix. Cependant, le modèle peut être amélioré pour mieux expliquer cette variance et réduire les erreurs.

Marge d'amélioration : Bien que l'erreur soit acceptable pour certaines applications (9.3% d'erreur sur des exemples d'annonces), des améliorations peuvent être apportées en ajustant les hyperparamètres, en utilisant d'autres modèles (par exemple, des modèles plus complexes ou non-linéaires), ou en ajoutant de nouvelles caractéristiques aux données.

Validation croisée solide : La validation croisée suggère que le modèle est robuste et qu'il généralise bien, ce qui est un bon signe pour sa stabilité.

Limites du Modèle


Précision modérée (R² de 0.505) :

Le modèle explique seulement 50.5% de la variance des prix, ce qui indique que plus de 40% des variations de prix ne sont pas capturées par le modèle. Cela montre une marge d'amélioration importante pour mieux comprendre les facteurs influençant le prix.

Erreur relativement élevée sur les prix réels (RMSE de 0.67 €) :

L'erreur quadratique moyenne de 0.67 € sur les prix réels peut être jugée acceptable, mais elle montre que le modèle a encore du mal à prédire précisément certains prix. Cela peut être problématique dans des contextes où une grande précision est requise (par exemple, la recommandation de prix dans une plateforme de vente).

Caractéristiques limitées :



Les caractéristiques utilisées (latitude, longitude, type de chambre, nombre de nuits minimales, etc.) sont relativement simples et peuvent ne pas capturer tous les facteurs importants influençant le prix d'une annonce. Par exemple, des informations sur les images de l'annonce, les équipements offerts, la proximité de lieux populaires ou des informations sur l'hôte peuvent être pertinentes mais ne sont pas prises en compte ici.

Interaction entre les caractéristiques :

Le modèle actuel ne capture peut-être pas assez bien les interactions complexes entre les différentes caractéristiques. Par exemple, la combinaison du type de chambre avec la disponibilité pourrait influencer davantage le prix, mais ces interactions ne sont pas explicitement modélisées.

Surajustement possible dans certains modèles de base :

Bien que la validation croisée suggère que le modèle généralise bien, certains des modèles de base (comme le RandomForestRegressor) peuvent toujours être sensibles au surajustement si les hyperparamètres ne sont pas bien réglés.

Suggestions d'Amélioration


Ajout de nouvelles caractéristiques :

Caractéristiques liées aux images : Ajouter des informations sur les images des annonces (par exemple, la qualité, le nombre d'images) pourrait améliorer la précision des prédictions.

Caractéristiques géographiques supplémentaires : Par exemple, la proximité de points d'intérêt, comme des attractions touristiques ou des centres de transport, pourrait influencer le prix.

Caractéristiques sur l'hôte : Le nombre d'années d'activité de l'hôte, la réputation de l'hôte (note moyenne, nombre d'avis) pourrait avoir un impact sur le prix.

Exploration de modèles plus avancés :

Modèles non-linéaires complexes : Des modèles comme les réseaux de neurones (NN), XGBoost, ou CatBoost pourraient capturer des relations plus complexes entre les caractéristiques. Ces modèles sont souvent très performants sur des données comme celles-ci.

Modèles de type Gradient Boosting comme XGBoost, LightGBM, ou CatBoost : Ces modèles sont bien adaptés aux données tabulaires avec des interactions complexes et des relations non-linéaires.

Optimisation de l'architecture du Stacking :

Ajout d'autres modèles de base : Vous pourriez intégrer des modèles comme SVM, KNN, ou des réseaux de neurones comme modèles de base dans le stacking pour en améliorer les performances.

Réduction du nombre de modèles de base : Parfois, simplifier le stacking en réduisant le nombre de modèles de base peut également améliorer la performance, si ces modèles ne sont pas réellement complémentaires.



Optimisation des hyperparamètres :

Hyperparamètres des modèles de base : Effectuer une recherche plus exhaustive des hyperparamètres des modèles de base comme le RandomForestRegressor et le GradientBoostingRegressor peut améliorer les performances du stacking.

Utilisation de techniques comme le Bayesian Optimization pour rechercher de manière plus efficace les hyperparamètres du stacking et des modèles de base.



Feature Engineering :

Interactions entre les caractéristiques : Vous pourriez essayer de créer de nouvelles caractéristiques représentant des interactions entre les caractéristiques existantes. Par exemple, combiner les informations de latitude/longitude pour créer des clusters géographiques et voir comment ces clusters affectent le prix.

Transformation des variables : Certaines variables (comme le nombre de nuits minimales ou le nombre d'avis) peuvent bénéficier de transformations non linéaires ou d'encodages spécifiques (logarithmiques, par exemple).

Amélioration de la gestion des valeurs manquantes :



Bien que vous ayez déjà bien géré les valeurs manquantes, il peut être utile d'explorer des méthodes de prédiction des valeurs manquantes plus complexes, comme des modèles de prédiction pour les variables manquantes (par exemple, en utilisant un modèle de régression pour prédire les valeurs manquantes).


Utilisation de l'architecture "Ensemble" :

Envisagez des méthodes Ensemble plus avancées, comme Stacking combinée avec Boosting ou Bagging, pour obtenir de meilleurs résultats en combinant plusieurs types de modèles.



Suivi de la stabilité et de la robustesse des prédictions :

Analyser les erreurs par région géographique ou par type de logement pourrait vous donner des indications sur les cas où le modèle échoue. Par exemple, peut-être que le modèle sous-estime les prix dans certaines zones ou types de logement, ce qui pourrait guider des améliorations supplémentaires dans les caractéristiques du modèle.